In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import cv2

In [3]:
resnet50_match_distance = pd.read_csv('resnet50_matches_distance.csv')
resnet50_match_filename = pd.read_csv('resnet50_matches_filename.csv')

In [11]:
resnet50_match_distance.values

array([[16.87788785, 19.18706897, 19.46590945, ..., 55.22238864,
        55.69534702, 64.75160245],
       [20.866944  , 22.17866929, 25.93556445, ..., 57.82651857,
        59.03646086, 63.50801961],
       [25.18745396, 26.85310245, 28.30647615, ..., 61.39092923,
        62.57981622, 68.30292491],
       ...,
       [19.95715779, 22.68322613, 23.28842565, ..., 57.87640682,
        60.74417852, 64.13282344],
       [21.76125141, 22.17268638, 22.74561712, ..., 57.18769424,
        57.29742892, 65.93675947],
       [35.21623528, 35.65160739, 36.94054192, ..., 63.79817037,
        64.99044777, 70.01153644]])

In [5]:
train_i = []
test_i = []

train_xy = pd.read_csv('train.csv')
train_path = train_xy['id'].values
test_path = pd.read_csv('imagenames.csv')['id'].values

for f in train_path:
    i = cv2.imread('./train/' + f + '.jpg')
    train_i.append(i)
for f in test_path:
    i = cv2.imread('./test/' + f + '.jpg')
    test_i.append(i)

In [7]:
len(test_i)

1200

In [9]:
sift = cv2.SIFT_create()


for i,img in tqdm(enumerate(test_i)):
    _, des = sift.detectAndCompute(img,None)
    f = open(f'./test_kp1/test_kp{test_path[i]}.pckl','wb')
    pickle.dump(des,f)
    f.close()


1200it [02:23,  8.37it/s]


In [25]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

i = 0
match=[]
for test in tqdm(test_path):

    f = open(f'./test_kp1/test_kp{test}.pckl', 'rb')
    des_test = pickle.load(f)
    f.close()

    goods = []
    current_good = 0

    if des_test is None or len(des_test) < 2:
        match.append((test,resnet50_match_filename.loc[i][0],resnet50_match_filename.loc[i][1],resnet50_match_filename.loc[i][2]))
        continue

    for train in resnet50_match_filename.loc[i][:10]:
        f = open(f'./train_kp/train_kp{train}.pckl', 'rb')
        des_train = pickle.load(f)
        f.close()

        # Matching descriptor using KNN algorithm
        if des_train is None or len(des_train) < 2:
            goods.append(-1)
            continue
        matches = flann.knnMatch(des_test,des_train,k=2)

        # Store all good matches as per Lowe's Ratio test.
        good = len([m for m,n in matches if m.distance < 0.7*n.distance])
        goods.append(good)
    
    sorted_idx = np.argsort(goods)
    sorted_path = [resnet50_match_filename.loc[i][idx] for idx in sorted_idx]
    sorted_goods = [goods[idx] for idx in sorted_idx]
    
#     print(i)
#     print(sorted_goods)
#     print(sorted_idx)
#     print(sorted_path)
    match.append((test,sorted_path[-1],sorted_path[-2],sorted_path[-3]))
    i +=1
    

100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [09:23<00:00,  2.13it/s]


In [54]:
out = []
for img in no_match:
    index = np.argwhere(test_path==img)[0][0]
    resnet50_match_filename.loc[index][:3]
    coor = [train_xy[train_xy['id'] == filename] for filename in resnet50_match_filename.loc[index][:3]]
    result = pd.concat(coor)
    out.append([img, np.mean(result['x']), np.mean(result['y'])])

In [26]:
out = []
for i in range(len(match)):
    coor = [train_xy[train_xy['id']== match[i][k]] for k in [1,2,3]]
    result = pd.concat(coor)
    out.append([match[i][0],np.mean(result['x']),np.mean(result['y'])])
out_csv = pd.DataFrame(out, columns=['id','x','y'])
out_csv.to_csv('resnet50_sift_top1_20_out.csv',index=False)